In [1]:
!nvidia-smi


Tue Feb  3 16:26:35 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
os.listdir()



['.config', 'Test.csv', 'Train.csv', 'Val.csv', 'sample_data']

In [3]:
import pandas as pd

train = pd.read_csv("Train.csv")
val   = pd.read_csv("Val.csv")
test  = pd.read_csv("Test.csv")

print("Train shape:", train.shape)
print("Columns:", train.columns)
train.head(3)


Train shape: (12575, 2)
Columns: Index(['Data', 'Label'], dtype='object')


,Data,Label
0,মুগ্ধ হয়ে গেলাম মামু. আর তোমায় কি কমু. বলো তোম...,1
1,এই কুত্তার বাচ্চাদের জন্য দেশটা আজ এমন অবস্তায়...,2
2,ভাই আপনার কথাই যাদু রয়েছে,1


In [4]:
print("Train labels:", sorted(train["Label"].unique()))
print("Val labels:", sorted(val["Label"].unique()))
print("Test labels:", sorted(test["Label"].unique()))


Train labels: [np.int64(0), np.int64(1), np.int64(2)]
Val labels: [np.int64(0), np.int64(1), np.int64(2)]
Test labels: [np.int64(0), np.int64(1), np.int64(2)]


In [5]:
print(train["Label"].value_counts())


Label
1    5133
2    4548
0    2894
Name: count, dtype: int64


In [6]:
from transformers import pipeline

MODEL_NAME = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
clf = pipeline("text-classification", model=MODEL_NAME)

print(clf("আমি আজকে খুব খুশি"))
print(clf("খুব খারাপ সার্ভিস"))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

XLMRobertaForSequenceClassification LOAD REPORT from: cardiffnlp/twitter-xlm-roberta-base-sentiment
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


[{'label': 'positive', 'score': 0.7924331426620483}]
[{'label': 'negative', 'score': 0.8623208403587341}]


In [7]:
import pandas as pd

# Make a clean copy with standard column names
train_df = train.rename(columns={"Data":"text", "Label":"label"}).copy()
val_df   = val.rename(columns={"Data":"text", "Label":"label"}).copy()
test_df  = test.rename(columns={"Data":"text", "Label":"label"}).copy()

# Convert label to int
train_df["label"] = train_df["label"].astype(int)
val_df["label"]   = val_df["label"].astype(int)
test_df["label"]  = test_df["label"].astype(int)

train_df.head(3)


,text,label
0,মুগ্ধ হয়ে গেলাম মামু. আর তোমায় কি কমু. বলো তোম...,1
1,এই কুত্তার বাচ্চাদের জন্য দেশটা আজ এমন অবস্তায়...,2
2,ভাই আপনার কথাই যাদু রয়েছে,1


In [8]:
min_label = min(train_df["label"].unique())
print("Min label:", min_label)


Min label: 0


In [9]:
!pip -q install -U transformers accelerate datasets evaluate


In [10]:
!pip -q install --upgrade --no-cache-dir \
  "pandas==2.2.2" \
  "requests==2.32.4"



In [11]:
!pip -q install --upgrade --no-cache-dir \
  "pyarrow>=21.0.0" \
  "datasets>=4.5.0" \
  "transformers" "accelerate" "evaluate" "scikit-learn"



In [12]:
import numpy as np, pandas as pd
import pyarrow as pa, datasets, transformers

print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("pyarrow:", pa.__version__)
print("datasets:", datasets.__version__)
print("transformers:", transformers.__version__)



numpy: 2.0.2
pandas: 2.2.2
pyarrow: 23.0.0
datasets: 4.5.0
transformers: 5.0.0


In [13]:
import torch
print("GPU available:", torch.cuda.is_available())
!nvidia-smi


GPU available: True
Tue Feb  3 16:36:27 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P0             28W /   70W |    1244MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------------------------

In [14]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [18]:
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate

MODEL_NAME = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

# number of classes from your data
NUM_LABELS = len(sorted(train_df["label"].unique()))
print("NUM_LABELS =", NUM_LABELS)

# ✅ define tokenizer first
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# ✅ then create collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def to_ds(df):
    df = df[["text","label"]].copy()
    df["text"] = df["text"].astype(str)
    df["label"] = df["label"].astype(int)
    return Dataset.from_pandas(df)

train_ds = to_ds(train_df)
val_ds   = to_ds(val_df)
test_ds  = to_ds(test_df)

def tok(batch):
    return tokenizer(batch["text"], truncation=True)

train_ds = train_ds.map(tok, batched=True)
val_ds   = val_ds.map(tok, batched=True)
test_ds  = test_ds.map(tok, batched=True)

train_ds.set_format("torch", columns=["input_ids","attention_mask","label"])
val_ds.set_format("torch", columns=["input_ids","attention_mask","label"])
test_ds.set_format("torch", columns=["input_ids","attention_mask","label"])

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=NUM_LABELS, ignore_mismatched_sizes=True
)

acc = evaluate.load("accuracy")
f1  = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": acc.compute(predictions=preds, references=labels)["accuracy"],
        "macro_f1": f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

args = TrainingArguments(
    output_dir="xlmr_finetuned_bn",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()
print("FINAL TEST:", trainer.evaluate(test_ds))



NUM_LABELS = 3


Map:   0%|          | 0/12575 [00:00<?, ? examples/s]

Map:   0%|          | 0/1567 [00:00<?, ? examples/s]

Map:   0%|          | 0/1586 [00:00<?, ? examples/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.output.LayerNorm.weight', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.1.output.LayerNorm.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.encoder.layer.3.output.LayerNorm.weight', 'roberta.encoder.layer.3.output.Laye

Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.759021,0.704292,0.684748,0.642149
2,0.585955,0.718114,0.717294,0.674408
3,0.396193,0.768183,0.721761,0.692697


FINAL TEST: {'eval_loss': 0.8018025159835815, 'eval_accuracy': 0.7131147540983607, 'eval_macro_f1': 0.6853093032869144, 'eval_runtime': 6.3271, 'eval_samples_per_second': 250.667, 'eval_steps_per_second': 31.452, 'epoch': 3.0}


In [19]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

pred = trainer.predict(test_ds)
y_pred = np.argmax(pred.predictions, axis=1)
y_true = pred.label_ids

print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred, digits=4))


Confusion Matrix:
 [[185  91  85]
 [ 96 511  47]
 [ 80  56 435]]

Classification Report:
               precision    recall  f1-score   support

           0     0.5125    0.5125    0.5125       361
           1     0.7766    0.7813    0.7790       654
           2     0.7672    0.7618    0.7645       571

    accuracy                         0.7131      1586
   macro avg     0.6854    0.6852    0.6853      1586
weighted avg     0.7131    0.7131    0.7131      1586



In [20]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Get predictions on test set
pred = trainer.predict(test_ds)
y_pred = np.argmax(pred.predictions, axis=1)
y_true = pred.label_ids  # safest (from dataset)

# Label names MUST match your numeric labels (0,1,2)
label_names = ["neutral", "positive", "negative"]   # change if your mapping is different

print(classification_report(y_true, y_pred, target_names=label_names, digits=4))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


              precision    recall  f1-score   support

     neutral     0.5125    0.5125    0.5125       361
    positive     0.7766    0.7813    0.7790       654
    negative     0.7672    0.7618    0.7645       571

    accuracy                         0.7131      1586
   macro avg     0.6854    0.6852    0.6853      1586
weighted avg     0.7131    0.7131    0.7131      1586

Confusion Matrix:
 [[185  91  85]
 [ 96 511  47]
 [ 80  56 435]]
